In [14]:
# Import libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori ,association_rules
import matplotlib.pyplot as plt
import seaborn as snsno
import datetime
import warnings

In [15]:
now = datetime.datetime.now(datetime.timezone.utc)
# Filter out the DeprecationWarning related to openpyxl
warnings.filterwarnings("ignore", category=DeprecationWarning)
#  Load data from an Excel file named "Basket_Cosmetic.xlsx"
df_sale_product=pd.read_excel("Basket_Cosmetic.xlsx")

In [16]:
#Check the shape (number of rows and columns) of the dataset
df_sale_product.shape

(4792, 10)

In [17]:
# Get information about the dataset, including data types and non-null counts
df_sale_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4792 entries, 0 to 4791
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   OrderId                4792 non-null   int64  
 1   OrderQty               4792 non-null   int64  
 2   SaleAmount             4792 non-null   float64
 3   CountOrder             4792 non-null   int64  
 4   AttributeSetName       4792 non-null   object 
 5   AttributeSetId         4792 non-null   int64  
 6   CategoryName_Level2    4792 non-null   object 
 7   ProductMiddleCategory  4792 non-null   object 
 8   ProductName            4792 non-null   object 
 9   ProductCode            4792 non-null   int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 374.5+ KB


In [18]:
# ProductRel222=ProductRel['SaleAmount']

In [19]:
# display the first two rows and the last row of the data to get an overview.
df_sale_product.head(2)
df_sale_product.tail(2)

,OrderId,OrderQty,SaleAmount,CountOrder,AttributeSetName,AttributeSetId,CategoryName_Level2,ProductMiddleCategory,ProductName,ProductCode
4790,376492,1,140000.0,1,آرايشي و بهداشتي,17,لوازم آرایشی,آرايشي و بهداشتي,Women's mom stick,529575
4791,372916,1,112000.0,1,آرايشي و بهداشتي,17,لوازم آرایشی,آرايشي و بهداشتي,Women's mom stick,529575


In [20]:
# Generate descriptive statistics for the dataset, including count, mean, min, max, etc.
df_sale_product.describe(include = 'all')

,OrderId,OrderQty,SaleAmount,CountOrder,AttributeSetName,AttributeSetId,CategoryName_Level2,ProductMiddleCategory,ProductName,ProductCode
count,4792.000000,4792.000000,4.792000e+03,4792.000000,4792,4792.0,4792,4792,4792,4792.000000
unique,NaN,NaN,NaN,NaN,1,NaN,6,3,31,NaN
top,NaN,NaN,NaN,NaN,آرايشي و بهداشتي,NaN,لوازم آرایشی,آرايشي و بهداشتي,perfume,NaN
freq,NaN,NaN,NaN,NaN,4792,NaN,1985,4699,750,NaN
mean,366736.435309,1.137312,1.468123e+05,1.000835,NaN,17.0,NaN,NaN,NaN,469527.186770
std,8837.134977,0.735771,2.106541e+05,0.028883,NaN,0.0,NaN,NaN,NaN,96311.422678
min,352685.000000,0.000000,0.000000e+00,1.000000,NaN,17.0,NaN,NaN,NaN,253867.000000
25%,358701.000000,1.000000,4.400000e+04,1.000000,NaN,17.0,NaN,NaN,NaN,460150.000000
50%,366208.000000,1.000000,8.900000e+04,1.000000,NaN,17.0,NaN,NaN,NaN,498215.000000
75%,374290.000000,1.000000,1.780000e+05,1.000000,NaN,17.0,NaN,NaN,NaN,540831.000000


In [21]:
# Insert a new column named 'quantity' with a default value of 1 at position 10
df_sale_product.insert(10, 'quantity',1)

In [22]:
# Group and pivot the data to create a basket of products by OrderId
product_basket = (df_sale_product.groupby(['OrderId', 'ProductCode'])['quantity']).sum().unstack().reset_index().fillna(0).set_index('OrderId')

In [25]:
# Define a function to convert values to 0 if they are less than or equal to 0, and 1 if greater than or equal to 1
def convto0(x):
    if (x<=0):
        return 0
    if (x>=1):
        return 1
# Apply the conversion function to the product_basket dataset
Basket_sets = product_basket.apply(lambda x: x.map(convto0))
Basket_sets.head()

ProductCode,253867,257556,257758,357021,363706,411587,435483,435488,454273,460150,...,529253,529575,532667,540831,541787,541788,542097,542718,542785,543073
OrderId,,,,,,,,,,,,,,,,,,,,,
352685,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
352697,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352740,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
352743,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352761,0,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [26]:
# Use the Apriori algorithm to mine frequent itemsets from the basket dataset
frequent_itemsets = apriori(Basket_sets.astype('bool'), min_support = 0.0008, use_colnames = True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.011387,(253867),1
1,0.164294,(257556),1
2,0.052460,(257758),1
3,0.002847,(357021),1
4,0.015047,(363706),1
...,...,...,...
658,0.000813,"(501795, 541787, 532667, 542718)",4
659,0.000813,"(542785, 529146, 532667, 540831)",4
660,0.000813,"(542785, 542097, 529253, 542718)",4
661,0.000813,"(498215, 532667, 257556, 541787, 542718)",5


In [27]:
# Filter the itemsets where the length is greater than or equal to 1
lenght=frequent_itemsets['itemsets'].str.len()
Filter1=lenght>=1
frequent_itemsets[Filter1]

,support,itemsets,length
0,0.011387,(253867),1
1,0.164294,(257556),1
2,0.052460,(257758),1
3,0.002847,(357021),1
4,0.015047,(363706),1
...,...,...,...
658,0.000813,"(501795, 541787, 532667, 542718)",4
659,0.000813,"(542785, 529146, 532667, 540831)",4
660,0.000813,"(542785, 542097, 529253, 542718)",4
661,0.000813,"(498215, 532667, 257556, 541787, 542718)",5


In [28]:
# Generate association rules from the frequent itemsets based on support
association_rules_data = association_rules(frequent_itemsets, metric = 'support', min_threshold = 0.0008)
association_rules_data

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(253867),(257556),0.011387,0.164294,0.002033,0.178571,1.086899,0.000163,1.017381,0.080872
1,(257556),(253867),0.164294,0.011387,0.002033,0.012376,1.086899,0.000163,1.001002,0.095669
2,(253867),(411587),0.011387,0.058560,0.000813,0.071429,1.219742,0.000147,1.013858,0.182230
3,(411587),(253867),0.058560,0.011387,0.000813,0.013889,1.219742,0.000147,1.002537,0.191361
4,(518771),(253867),0.026027,0.011387,0.000813,0.031250,2.744420,0.000517,1.020504,0.652610
...,...,...,...,...,...,...,...,...,...,...
2911,(501539),"(529146, 541787, 542718, 498215)",0.017080,0.000813,0.000813,0.047619,58.547619,0.000799,1.049146,1.000000
2912,(498215),"(529146, 501539, 541787, 542718)",0.228955,0.000813,0.000813,0.003552,4.367673,0.000627,1.002749,1.000000
2913,(529146),"(541787, 501539, 542718, 498215)",0.042294,0.000813,0.000813,0.019231,23.644231,0.000779,1.018779,1.000000
2914,(541787),"(529146, 501539, 542718, 498215)",0.061814,0.000813,0.000813,0.013158,16.177632,0.000763,1.012509,1.000000


In [29]:
association_rules_data.dtypes

antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
zhangs_metric         float64
dtype: object

In [30]:
# Export the rules to an Excel file named 'BasketAnalysisOutput.xlsx'
excell=association_rules_data.to_excel('BasketAnalysisOutput.xlsx')